# Проект по выгрузке данных из базы с помощью SQL-зпросов

##### Содержание

* [1. Описание и цель проекта](#section_1)
* [2. Импорт библиотек и подключение к базе данных](#section_2)
* [3. Запрос: Количество книг, опубликованных после 01/01/2000](#section_3)
* [4. Запрос: Количество обзоров и средняя оценка для каждой книги](#section_4)
* [5. Запрос: Запрос: Издательство, которое выпустило наибольшее число книг толще 50 страниц](#section_5)
* [6. Запрос: Автор с самой высокой средней оценкой книг](#section_6)
* [7. Запрос: Cреднее количество обзоров от пользователей с большим количеством оценок](#section_7)

<a id = 'bullet_1'></a>
### 1. Описание и цель проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг.

Наша компания купила крупный сервис для чтения книг по подписке. Задача данного проекта — проанализировать базу данных,  в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. На основании этих данных планируется сформулировать ценностное предложение для нового продукта.

<a id = 'bullet_2'></a>
### 2. Импорт библиотек и подключение к базе данных

In [15]:
# импортируем библиотеки
import pandas  as pd
from sqlalchemy import create_engine

In [16]:
# устанавливаем параметры
db_config = {'user': '{ххххххххх}',
             'pwd': 'хххххххх',
             'host': 'ххххххххххххх.mdb.yandexcloud.net',
             'port': хххх,
             'db': 'хххххххххххххххххххх'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args = {'sslmode': 'require'})

In [17]:
# просматриваем первые строки таблиц и типы данных
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for table in tables:
    query1 = 'SELECT * FROM ' + table  + ' LIMIT 5'
    query2 = """
    SELECT 
        table_name, 
        column_name, 
        data_type 
    FROM information_schema.columns 
    WHERE table_name = '%s'"""
    query3 = 'SELECT COUNT(*) FROM ' + table
    display(table)
    display(pd.io.sql.read_sql(query1, con = engine))
    display(pd.io.sql.read_sql(query2 % table, con = engine))
    display(pd.io.sql.read_sql(query3, con = engine))

'books'

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,table_name,column_name,data_type
0,books,book_id,integer
1,books,author_id,integer
2,books,title,text
3,books,num_pages,integer
4,books,publication_date,date
5,books,publisher_id,integer


,count
0,1000


'authors'

...

,count
0,6456


'reviews'

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,table_name,column_name,data_type
0,reviews,review_id,integer
1,reviews,book_id,integer
2,reviews,username,text
3,reviews,text,text


,count
0,2793


В нашей базе имеется 5 таблиц: таблица с книгами (с указанием названия книги, даты ее публикации, количества страниц), таблица с авторами, таблица с издательствами, с оценками и с обзорами.

Мы также посмотрели типы данных для столбцов в каждой таблице и не выявили никаких несоответствий.

In [18]:
# напишем функцию для выгрузки результатов SQL-запроса через pandas
def request(query):
    display(pd.io.sql.read_sql(query, con = engine))

<a id = 'section_3'></a> 
###  3. Запрос: Количество книг, опубликованных после 01/01/2000

Напишем запрос, чтобы узнать, сколько книг вышло после 1 января 2000 года.

In [19]:
request(
'''
SELECT COUNT (*) 
FROM 
    books 
WHERE
    publication_date > '2000-01-01'
''')

,count
0,819


In [20]:
request(
'''
SELECT COUNT (*)
FROM books
''')

,count
0,1000


В нашей базе данных содержится информация о 1000 книг, 819 из которых были опубликованы после 1 января 2000 года. Таким образом, можно сделать вывод, что большая часть книг в нашей базе современные. 

<a id = 'section_4'></a> 
### 4. Запрос: Количество обзоров и средняя оценка для каждой книги

Напишем запрос, чтобы для каждой книги посчитать количество обзоров и среднюю оценку.

In [21]:
request(
'''
SELECT
    books.book_id,
    books.title,
    ROUND(rating_table.avg_rating, 2) AS avg_rating,
    review_table.no_of_reviews
FROM
    books
LEFT JOIN
    (SELECT
        ratings.book_id,
        AVG(ratings.rating) AS avg_rating
    FROM
        ratings
    GROUP BY ratings.book_id) AS rating_table ON books.book_id = rating_table.book_id
LEFT JOIN
    (SELECT
        reviews.book_id,
        COUNT(reviews.review_id) AS no_of_reviews
    FROM
        reviews
    GROUP BY reviews.book_id) AS review_table ON books.book_id = review_table.book_id
ORDER BY
    avg_rating DESC
'''
)

,book_id,title,avg_rating,no_of_reviews
0,518,Pop Goes the Weasel (Alex Cross #5),5.00,2.0
1,732,The Ghost Map: The Story of London's Most Terr...,5.00,2.0
2,347,In the Hand of the Goddess (Song of the Liones...,5.00,2.0
3,610,Tai-Pan (Asian Saga #2),5.00,2.0
4,330,How to Be a Domestic Goddess: Baking and the A...,5.00,1.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,371,Junky,2.00,2.0
997,316,His Excellency: George Washington,2.00,2.0
998,202,Drowning Ruth,2.00,3.0


In [22]:
request(
'''
SELECT
    COUNT (*)
FROM
    books
RIGHT JOIN
    (SELECT
        ratings.book_id,
        AVG(ratings.rating) AS avg_rating
    FROM
        ratings
    GROUP BY ratings.book_id
    HAVING AVG(ratings.rating) >= 4) AS rating_table ON books.book_id = rating_table.book_id
LEFT JOIN
    (SELECT
        reviews.book_id,
        COUNT(reviews.review_id) AS no_of_reviews
    FROM
        reviews
    GROUP BY reviews.book_id) AS review_table ON books.book_id = review_table.book_id
'''
)

,count
0,564


Мы посчитали количество обзоров и среднюю оценку для каждой книги. Также мы посмотрели сколько книг в нашей базе имеют среднюю оценку больше 4 - таких книг у нас 564, то есть больше половины (56,4%).

<a id = 'section_5'></a> 
###  5. Запрос: Издательство, которое выпустило наибольшее число книг толще 50 страниц

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [23]:
request(
'''
SELECT 
    publishers.publisher,
    COUNT(thick_books.title) AS books_cnt
FROM 
    publishers
LEFT JOIN 
    (SELECT *
    FROM 
        books
    WHERE 
        num_pages :: integer > 50) AS thick_books ON publishers.publisher_id = thick_books.publisher_id 
GROUP BY
    publishers.publisher
ORDER BY
    books_cnt DESC
LIMIT 1'''
)

,publisher,books_cnt
0,Penguin Books,42


Издательство с наибольшим количеством книг более 50 страниц в нашей базе - это британское издательство Penguin Books. Ему принадлежит 42 книги.

<a id = 'section_6'></a> 
###  6. Запрос: Автор с самой высокой средней оценкой книг

Определим автора с самой высокой средней оценкой книг; при этом будем учитывать только книги с 50 и более оценками.

In [24]:
request('''
SELECT
    authors.author,
    ROUND(rating_table.avg_rating, 2) AS rating
FROM
    books
JOIN
    authors
ON books.author_id = authors.author_id
RIGHT JOIN
    (SELECT
         book_id,
         COUNT(rating_id) AS ratings_cnt,
         AVG(rating) AS avg_rating
     FROM
         ratings
     GROUP BY
         book_id
     HAVING
         COUNT(rating_id) > 50) AS rating_table ON rating_table.book_id = books.book_id
ORDER BY 
    rating_table.avg_rating DESC

''')

,author,rating
0,J.K. Rowling/Mary GrandPré,4.41
1,J.R.R. Tolkien,4.39
2,J.K. Rowling/Mary GrandPré,4.29
3,Markus Zusak/Cao Xuân Việt Khương,4.26
4,J.K. Rowling/Mary GrandPré,4.25
5,Louisa May Alcott,4.19
6,J.K. Rowling/Mary GrandPré,4.19
7,J.R.R. Tolkien,4.13
8,Rick Riordan,4.08
9,William Golding,3.90


In [33]:
request('''
SELECT
    authors.author,
    AVG(rating_table.book_avg_rating) AS author_avg_rating
FROM
    books
JOIN
    authors
ON books.author_id = authors.author_id
RIGHT JOIN
    (SELECT
         book_id,
         COUNT(rating_id) AS ratings_cnt,
         AVG(rating) AS book_avg_rating
     FROM
         ratings
     GROUP BY
         book_id
     HAVING
         COUNT(rating_id) > 50) AS rating_table ON rating_table.book_id = books.book_id
GROUP BY 
    authors.author
ORDER BY 
    author_avg_rating DESC'''
       )

,author,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самую высокую среднюю оценку книг, среди книг с 50 и более оценками, имеет Джоан Роулинг в соавторстве с иллюстратором Мэри Грандпре - судя по всему, речь идет о книгах о Гарри Поттере. Средняя оценка книг данных авторов - 4.41.

<a id = 'section_7'></a> 
###  7. Запрос: Cреднее количество обзоров от пользователей с большим количеством оценок

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
request('''
SELECT
    ROUND(AVG(reviews_table.reviews_qty), 0) AS avg_number_of_reviews
FROM
    (SELECT 
        username,
        COUNT(rating) AS rating_cnt
    FROM 
        ratings
    GROUP BY 
        username
    HAVING
        COUNT(rating) > 50) AS rating_table
LEFT JOIN
    (SELECT
         username,
         COUNT(text) as reviews_qty
     FROM
         reviews
     GROUP BY
         username) AS reviews_table ON rating_table.username = reviews_table.username
''')

,avg_number_of_reviews
0,24.0


Пользователи, которые поставили больше 50 оценок, в среднем написали по 24 отзыва.